### 可见性和属性装饰器

一般来说，对象的属性会是private 或者 protected，就是不允许直接访问，对象的方法一般来说都是 public

在 Python 中，可以通过给对象属性名添加前缀下划线的方式来说明属性的访问可见性，例如，可以用 `__name` 表示一个私有属性，`_name` 表示一个受保护属性，代码如下所示。

In [2]:
class Student:
    def __init__(self, name, age):
        self.__name = name
        self.__age = age
    
    def study(self, course_name):
        print(f"{self.__name} is studying {course_name}")

stu = Student("Alice", 20)
stu.study("Python Programming")
# print(stu.__name)  # --- IGNORE ---

Alice is studying Python Programming


事实上，大多数的程序员都认为开放比封闭要好，把对象的属性私有化并非必不可少的东西，所以 Python 语言并没有从语义上做出最严格的限定，也就是说上面的代码如果你愿意，用stu._Student__name的方式仍然可以访问到私有属性__name，有兴趣的读者可以自己试一试。

### 动态属性

Python 语言属于动态语言，维基百科对动态语言的解释是：“在运行时可以改变其结构的语言，例如新的函数、对象、甚至代码可以被引进，已有的函数可以被删除或是其他结构上的变化”

在 Python 中，我们可以动态为对象添加属性

In [3]:
class Student:

    def __init__(self, name, age):
        self.name = name
        self.age = age


stu = Student('王大锤', 20)
stu.sex = '男'  # 给学生对象动态添加sex属性

如果不希望在使用对象时动态的为对象添加属性，可以使用 Python 语言中的 `__slots__` 魔法。对于Student类来说，可以在类中指定 `__slots__ = ('name', 'age')`，这样Student类的对象只能有name和age属性，如果想动态添加其他属性将会引发异常，代码如下所示。

In [4]:
class Student:
    __slots__ = ('name', 'age')

    def __init__(self, name, age):
        self.name = name
        self.age = age


stu = Student('王大锤', 20)
# AttributeError: 'Student' object has no attribute 'sex'
stu.sex = '男'

AttributeError: 'Student' object has no attribute 'sex'

### 静态方法和类方法

之前我们在类中定义的方法都是对象方法，换句话说这些方法都是对象可以接收的消息。除了对象方法之外，类中还可以有静态方法和类方法，这两类方法是发给类的消息，二者并没有实质性的区别。在面向对象的世界里，一切皆为对象，我们定义的每一个类其实也是一个对象，而静态方法和类方法就是发送给类对象的消息。那么，什么样的消息会直接发送给类对象呢？

举一个例子，定义一个三角形类，通过传入三条边的长度来构造三角形，并提供计算周长和面积的方法。计算周长和面积肯定是三角形对象的方法，这一点毫无疑问。但是在创建三角形对象时，传入的三条边长未必能构造出三角形，为此我们可以先写一个方法来验证给定的三条边长是否可以构成三角形，这种方法很显然就不是对象方法，因为在调用这个方法时三角形对象还没有创建出来。我们可以把这类方法设计为静态方法或类方法，也就是说这类方法不是发送给三角形对象的消息，而是发送给三角形类的消息，代码如下所示。

In [11]:
class Triangle(object):
    """三角形"""
    def __init__(self, a, b, c):
        if not Triangle.is_valid(a, b, c):
            raise ValueError("不能构成三角形的边长")
        
        self.a = a
        self.b = b
        self.c = c
    
    @staticmethod
    def is_valid(a, b, c):
        """判断三边长能否构成三角形(静态方法)"""
        return a + b > c and a + c > b and b + c > a

    # @classmethod
    # def is_valid(cls, a, b, c):
    #     """判断三边长能否构成三角形（类方法）"""
    #     return a + b > c and a + c > b and b + c > a
    
    def perimeter(self):
        """计算周长"""
        return self.a + self.b + self.c

    def area(self):
        """计算面积"""
        p = self.perimeter() / 2
        return (p * (p - self.a) * (p - self.b) * (p - self.c)) ** 0.5

# 情况 1：合法的三角形
t = Triangle(3, 4, 5) # 内部检查通过，创建成功
print("创建成功")

# 情况 2：非法的三角形
try:
    # 这一行执行时，__init__ 会直接报错，阻止垃圾对象产生
    bad_t = Triangle(1, 1, 10) 
except ValueError as e:
    print(f"创建失败: {e}")

创建成功
创建失败: 不能构成三角形的边长


二者的区别在于，类方法的第一个参数是类对象本身，而静态方法则没有这个参数。

简单的总结一下，对象方法、类方法、静态方法都可以通过“类名.方法名”的方式来调用，区别在于方法的第一个参数到底是普通对象还是类对象，还是没有接受消息的对象。静态方法通常也可以直接写成一个独立的函数，因为它并没有跟特定的对象绑定。

我们可以给上面计算三角形周长和面积的方法添加一个property装饰器（Python 内置类型），这样三角形类的perimeter和area就变成了两个属性，不再通过调用方法的方式来访问，而是用对象访问属性的方式直接获得，修改后的代码如下所示。

In [6]:
class Triangle(object):
    """三角形"""

    def __init__(self, a, b, c):
        """初始化方法"""
        self.a = a
        self.b = b
        self.c = c

    @staticmethod
    def is_valid(a, b, c):
        """判断三条边长能否构成三角形(静态方法)"""
        return a + b > c and b + c > a and a + c > b

    @property
    def perimeter(self):
        """计算周长"""
        return self.a + self.b + self.c

    @property
    def area(self):
        """计算面积"""
        p = self.perimeter / 2
        return (p * (p - self.a) * (p - self.b) * (p - self.c)) ** 0.5


t = Triangle(3, 4, 5)
print(f'周长: {t.perimeter}')
print(f'面积: {t.area}')

周长: 12
面积: 6.0


### 继承和多态

面向对象的编程语言支持在已有类的基础上创建新类，从而减少重复代码的编写。提供继承信息的类叫做父类（超类、基类），得到继承信息的类叫做子类（派生类、衍生类）。例如，我们定义一个学生类和一个老师类，我们会发现他们有大量的重复代码，而这些重复代码都是老师和学生作为人的公共属性和行为，所以在这种情况下，我们应该先定义人类，再通过继承，从人类派生出老师类和学生类，代码如下所示。

In [12]:
class Person:
    """人"""

    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def eat(self):
        print(f'{self.name}正在吃饭.')
    
    def sleep(self):
        print(f'{self.name}正在睡觉.')


class Student(Person):
    """学生"""
    
    def __init__(self, name, age):
        super().__init__(name, age)
    
    def study(self, course_name):
        print(f'{self.name}正在学习{course_name}.')


class Teacher(Person):
    """老师"""

    def __init__(self, name, age, title):
        super().__init__(name, age)
        self.title = title
    
    def teach(self, course_name):
        print(f'{self.name}{self.title}正在讲授{course_name}.')



stu1 = Student('白元芳', 21)
stu2 = Student('狄仁杰', 22)
tea1 = Teacher('武则天', 35, '副教授')
stu1.eat()
stu2.sleep()
tea1.eat()
stu1.study('Python程序设计')
tea1.teach('Python程序设计')
stu2.study('数据科学导论')

白元芳正在吃饭.
狄仁杰正在睡觉.
武则天正在吃饭.
白元芳正在学习Python程序设计.
武则天副教授正在讲授Python程序设计.
狄仁杰正在学习数据科学导论.


### 总结

Python 是动态类型语言，Python 中的对象可以动态的添加属性，对象的方法其实也是属性，只不过和该属性对应的是一个可以调用的函数。在面向对象的世界中，一切皆为对象，我们定义的类也是对象，所以类也可以接收消息，对应的方法是类方法或静态方法。通过继承，我们可以从已有的类创建新类，实现对已有类代码的复用。